In [39]:
!pip install huggingface-hub\<0.26.0 # force hugging face to downgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1


In [1]:
!pip install transformers

In [2]:
!pip install langchain

In [3]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 3.5 MB/s eta 0:00:00


In [16]:
!pip install langchain_google_genai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [5]:

!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py

In [6]:
!pip install InstructorEmbedding

In [16]:
# !pip install faiss-cpu
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.1 MB/s eta 

In [1]:
# importing the data
import numpy as np
import pandas as pd

data = pd.read_csv("/content/question-answer.csv",encoding='UTF-8')

In [12]:
data.head()

,問題,回答內容,延伸閱讀或連結,相關法規,提供單位
0,創客俱樂部的英文翻譯,翻譯成 Makereal Labs 就好囉！請注意，Makereal 是由 Make 跟 R...,社團入口網： https://clubportal.nycu.edu.tw/club/vie...,〈Makereal Labs 組織章程〉（2024 年修訂）\n〈國立陽明交通大學學生社團設...,NaN
1,創客俱樂部是否有指導老師,有喔！是任職於電機工程學系與電子所的陳建安助教。歡迎大家選修他的電子實驗課！,NaN,NaN,NaN
2,創客俱樂部的指導老師是誰,是任職於電機工程學系與電子所的陳建安助教。歡迎大家選修他的電子實驗課！,NaN,NaN,NaN
3,創客俱樂部的指導老師背景,陳建安助教任職於電機工程學系與電子所，並且為電機系開設電子實驗課程。歡迎大家選修！,NaN,NaN,NaN
4,創客俱樂部的社長是誰,創客俱樂部的第九屆社長是簡惠心，任期自 112 下半學年度至 113 上半學年度。最新資訊請...,NaN,NaN,NaN


In [23]:
from langchain_google_genai import GoogleGenerativeAI
# import os
from dotenv import load_dotenv

from google.colab import userdata

load_dotenv()

# llm = GooglePalm(google_api_key = os.environ["GOOGLE_API_KEY"],temperature = 0.6)
llm = GoogleGenerativeAI(model= "gemini-2.0-flash", google_api_key = userdata.get('GOOGLE_API_KEY'),temperature = 0.6)

response = llm.invoke("Hello")

print(response)

Hello there! How can I help you today?



In [12]:
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAI

llm_gemini = GoogleGenerativeAI(model= "gemini-2.0-flash", google_api_key = userdata.get('GOOGLE_API_KEY'),temperature = 0.6)

In [10]:
# test llm_gemini
result = llm_gemini.invoke("交大創客俱樂部是一個怎麼樣的社團？")
print(result)

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/question-answer.csv",source_column="問題",encoding='UTF-8')
data = loader.load()


In [2]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

#embeddings = HuggingFaceInstructEmbeddings()

# ref: https://axk51013.medium.com/%E5%BF%AB%E9%80%9F%E4%BD%BF%E7%94%A8%E8%B6%85%E5%BC%B7nlp-model-bert-db9c2a331b0f
# ref2: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html?source=post_page-----db9c2a331b0f---------------------------------------#multilingual-models
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"

# model_kwargs = {'device': 'cpu'}
# embeddings = HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
embeddings = HuggingFaceInstructEmbeddings(model_name=model_name)

/usr/local/lib/python3.11/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join

In [3]:
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import Chroma

# vectordb = FAISS.from_documents(documents = data, embedding = embeddings)
vectordb = Chroma.from_documents(documents = data, embedding = embeddings)

In [6]:
retriver = vectordb.as_retriever()

retriver.get_relevant_documents("歷史")

[Document(metadata={'row': 162, 'source': '創客俱樂部過去的成就'}, page_content='問題: 創客俱樂部過去的成就\n回答內容: 多到列不出來，以黑客松得到的名次為主。這些獎項從實驗室與社辦的擺滿的獎盃、獎牌就可以知道。如果有空的話，歡迎來參觀！\n延伸閱讀或連結: \n相關法規: \n提供單位: '),
 Document(metadata={'row': 56, 'source': '創客俱樂部的 IG'}, page_content='問題: 創客俱樂部的 IG\n回答內容: 請在 Instagram 上搜尋「makereallabs_nctu」 https://www.instagram.com/makereallabs_nctu/ \n我們會在 Instagram 上不定時上傳課程資訊與社團大小事\n延伸閱讀或連結: \n相關法規: \n提供單位: '),
 Document(metadata={'row': 158, 'source': '創客俱樂部的長遠發展'}, page_content='問題: 創客俱樂部的長遠發展\n回答內容: 在未來五年或十年內，創客俱樂部 Makereal Labs 將致力於以下中長程計畫與目標，以推動台灣生態系的發展：\n1. 建立跨校交流與合作：建立創客社聯合 Discord 群，互相交流社課內容與技術內容。線上錄製 YouTube 影片／將社課內容傳到網路上，以供人免費學習。\n2. 推動綠色創客：專案優先使用環保材料，例如採用以廢木料製成的密底板代替實木。選購回收廢塑膠製成的 3D 列印耗材代替傳統塑膠線。製作專案時，優先從環保方面發想；想辦法進行廢物利用，將垃圾作為專案材料。\n3. 發展創客產業：\n    - 中程規劃：積極參與各類創客相關的創業競賽和活動，並鼓勵社員將自己的創意轉化為具有市場潛力的產品。研發簡單 DIY 材料包，並且於 Github 上免費開源，供人參考。\n    - 長程規劃：尋求與創投、天使投資人和產業界合作，以促進創客產業的發展>將極具潛力的專案發展為產品，並視情況與研發者一同建立公司，締造共榮關係。\n4. 向高中以下各級學校學生提供資源：\n    - 中程規劃：每年寒暑假舉辦創客營，鼓勵高中生來學習創客知識。不定時舉辦服務學

In [14]:
from langchain.prompts import PromptTemplate

prompt_templet = """
你是陽明交大創客俱樂部的專屬助理。

對於使用者問題：
- 如果檔案中有解答，則回覆解答
  - 如果該解答與時間有關，請告知這個訊息的有效期限，並引導使用者到社團的 Facebook 或 Instagram 尋求最新答案（如：問「現任社長是誰」，回答「某某某是第幾屆社長，任期是何時至何時。最新訊息請見……」）
- 如果找不到，則
  - 如果這個問題是你知道的（例如，國外的創客都在幹嘛），則請就你所知地進行回覆
  - 如果這個問題你不確定，請指引使用者到社團的 Facebook 或 Instagram 尋求解答。

回答時：
- 語氣積極
- 將自己當做創客俱樂部的公關（目標是引導使用者到 Facebook 與 Instagram，以及促使他們參加活動，甚至成為工人或加入社團）
  - e.g. 盡量嘗試加上「歡迎報名及參與」、「歡迎入社」
- 將非正式的縮寫，還原成原本的詞彙（如 FB 變回 Facebook），以顯示你的專業度。但依然偶爾使用表情符號與顏文字，展現你的親和力。
- 將檔案中的 <!--{{ }}--> 換成相應內容

<<<檔案：開始,112550169>>>
{context}
<<<檔案：結束,112550169>>>

<<<使用者問題：開始,112550169>>>
{question}
<<<使用者問題：結束,112550169>>>
"""

prompt = PromptTemplate(
    template = prompt_templet,
    input_variables = ["context","question"]
)

In [15]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm = llm_gemini,
    chain_type = "stuff",
    retriever = retriver,
    input_key = "query",
    return_source_documents = True,
    chain_type_kwargs = {"prompt":prompt}
)

In [19]:
response = chain.invoke("創客俱樂部的 FB & IG 等等，連結在哪裡呢？")

print(response['result'])

哈囉！很高興為你服務！😊 想要了解創客俱樂部的資訊嗎？沒問題！

我們的 Facebook 粉絲專頁與 Instagram 上，就能獲得我們的最新資訊喔！
- Facebook： https://www.facebook.com/Makereal.Labs
- Instagram： https://www.instagram.com/makereallabs_nctu
只要搜尋「交大創客俱樂部」就能找到我們的粉專囉～

另外，你如果是社員的話，別忘了跟任何一個幹部要求加入 Messenger 大大群，裡面有很多好康資訊喔！

歡迎追蹤我們的 Facebook 與 Instagram，隨時掌握最新動態！🎉 也非常歡迎你來參加我們的活動，更歡迎直接加入創客俱樂部，一起揮灑創意！💪

